In [2]:
import pandas as pd
import requests
import json

url = "https://gbfs.bluebikes.com/gbfs/gbfs.json"
r = requests.get(url)

print(json.dumps(r.json(), indent = 4, sort_keys=True))

{
    "data": {
        "en": {
            "feeds": [
                {
                    "name": "system_information",
                    "url": "https://gbfs.bluebikes.com/gbfs/en/system_information.json"
                },
                {
                    "name": "station_information",
                    "url": "https://gbfs.bluebikes.com/gbfs/en/station_information.json"
                },
                {
                    "name": "station_status",
                    "url": "https://gbfs.bluebikes.com/gbfs/en/station_status.json"
                },
                {
                    "name": "free_bike_status",
                    "url": "https://gbfs.bluebikes.com/gbfs/en/free_bike_status.json"
                },
                {
                    "name": "system_hours",
                    "url": "https://gbfs.bluebikes.com/gbfs/en/system_hours.json"
                },
                {
                    "name": "system_calendar",
                    "url": 

In [88]:
url = "https://gbfs.bluebikes.com/gbfs/en/station_information.json"
r = requests.get(url)

print(json.dumps(r.json(), indent = 4, sort_keys=True))


{
    "data": {
        "stations": [
            {
                "capacity": 15,
                "eightd_has_key_dispenser": true,
                "eightd_station_services": [],
                "electric_bike_surcharge_waiver": false,
                "external_id": "f83464e4-0de8-11e7-991c-3863bb43a7d0",
                "has_kiosk": true,
                "lat": 42.34011512249236,
                "legacy_id": "3",
                "lon": -71.10061883926392,
                "name": "Colleges of the Fenway - Fenway at Avenue Louis Pasteur",
                "region_id": "10",
                "rental_methods": [
                    "CREDITCARD",
                    "KEY"
                ],
                "short_name": "B32006",
                "station_id": "3",
                "station_type": "classic"
            },
            {
                "capacity": 19,
                "eightd_has_key_dispenser": false,
                "eightd_station_services": [],
                "electric_bi

In [93]:
station_information = r.json()["data"]["stations"]

df = pd.read_json(json.dumps(station_information))

df.head(1)

,lon,has_kiosk,short_name,region_id,legacy_id,eightd_station_services,electric_bike_surcharge_waiver,station_type,capacity,eightd_has_key_dispenser,rental_methods,lat,external_id,station_id,name
0,-71.100619,True,B32006,10.0,3,[],False,classic,15,True,"[CREDITCARD, KEY]",42.340115,f83464e4-0de8-11e7-991c-3863bb43a7d0,3,Colleges of the Fenway - Fenway at Avenue Loui...


In [103]:
def clean_stationdata(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes the datatypes for each column."""

    # time_prepared
    df.loc[:, "time_prepared"] = datetime.now()

    # stationname
    df.loc[:, "station_name"] = df.loc[:, "name"].astype("string")
    df.station_name = df.station_name.str.lower()
    df = df.drop(["name"], axis=1)

    # eightd_has_key_dispenser
    df = df.drop(["eightd_has_key_dispenser"], axis=1)

    # rental_methods
    df = df.drop(["rental_methods"], axis=1)

    # region_id
    df.region_id = df.region_id.values.astype("int8")

    # eightd_station_services
    df = df.drop(["eightd_station_services"], axis=1)

    # electric_bike_surcharge_waiver
    df = df.drop(["electric_bike_surcharge_waiver"], axis=1)

    # station_type
    df.station_type = df.station_type.str.lower()
    df.station_type = df.station_type.astype("category")

    # short_name
    df.short_name = df.short_name.astype("string")

    # legacy_id
    df = df.drop(["legacy_id"], axis=1)

    # station_id
    df.station_id = df.station_id.values.astype("int16")

    # has_kiosk
    df = df.drop(["has_kiosk"], axis=1)

    # external_id
    df = df.drop(["external_id"], axis=1)

    # capacity
    df.capacity = df.capacity.values.astype("int8")

    # station_longitude
    df.loc[:, "station_longitude"] = df.loc[:, "lon"].astype("float64")
    df = df.drop(["lon"], axis=1)

    # station_latitude
    df.loc[:, "station_latitude"] = df.loc[:, "lat"].astype("float64")
    df = df.drop(["lat"], axis=1)

    return df

df_cleaned = clean_stationdata(df)
df_cleaned.dtypes

/tmp/ipykernel_1462016/974063955.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "station_name"] = df.loc[:, "name"].astype("string")
/tmp/ipykernel_1462016/974063955.py:16: RuntimeWarning: invalid value encountered in cast
  df.region_id = df.region_id.values.astype("int8")


short_name             string
region_id                int8
station_type         category
capacity                 int8
station_id              int16
station_name           string
station_longitude     float64
station_latitude      float64
dtype: object

In [94]:
df.dtypes

lon                               float64
has_kiosk                            bool
short_name                         object
region_id                         float64
legacy_id                           int64
eightd_station_services            object
electric_bike_surcharge_waiver       bool
station_type                       object
capacity                            int64
eightd_has_key_dispenser             bool
rental_methods                     object
lat                               float64
external_id                        object
station_id                          int64
name                               object
dtype: object

In [102]:
df.station_id.value_counts()

3      1
402    1
432    1
431    1
430    1
      ..
176    1
175    1
174    1
173    1
593    1
Name: station_id, Length: 453, dtype: int64

In [113]:
from datetime import datetime, timedelta
f"{datetime.now().year:4}_{datetime.now().month:02}_{datetime.now().day:02}"

'2023_03_03'

In [138]:
from datetime import datetime, timedelta
url = "https://gbfs.bluebikes.com/gbfs/en/station_status.json"
r = requests.get(url)

print(json.dumps(r.json(), indent = 4, sort_keys=True))

{
    "data": {
        "stations": [
            {
                "eightd_has_available_keys": false,
                "is_installed": 1,
                "is_renting": 1,
                "is_returning": 1,
                "last_reported": 1677849887,
                "legacy_id": "3",
                "num_bikes_available": 3,
                "num_bikes_disabled": 0,
                "num_docks_available": 12,
                "num_docks_disabled": 0,
                "num_ebikes_available": 0,
                "station_id": "3",
                "station_status": "active"
            },
            {
                "eightd_has_available_keys": false,
                "is_installed": 1,
                "is_renting": 1,
                "is_returning": 1,
                "last_reported": 1677850067,
                "legacy_id": "4",
                "num_bikes_available": 7,
                "num_bikes_disabled": 0,
                "num_docks_available": 12,
                "num_docks_disabled":

In [139]:
station_status = r.json()["data"]["stations"]

df = pd.read_json(json.dumps(station_status))

df.head(3)

,station_status,is_renting,eightd_has_available_keys,num_docks_available,is_returning,is_installed,num_docks_disabled,num_bikes_disabled,station_id,num_bikes_available,num_ebikes_available,last_reported,legacy_id
0,active,1,False,12,1,1,0,0,3,3,0,1677849887,3
1,active,1,False,12,1,1,0,0,4,7,0,1677850067,4
2,out_of_service,0,False,0,0,0,0,0,5,0,0,86400,5


In [140]:
df.dtypes

station_status               object
is_renting                    int64
eightd_has_available_keys      bool
num_docks_available           int64
is_returning                  int64
is_installed                  int64
num_docks_disabled            int64
num_bikes_disabled            int64
station_id                    int64
num_bikes_available           int64
num_ebikes_available          int64
last_reported                 int64
legacy_id                     int64
dtype: object

In [141]:
def clean_station_status(df: pd.DataFrame) -> pd.DataFrame:
    """Optimizes the datatypes for each column."""

    # time_prepared
    df.loc[:, "time_prepared"] = datetime.utcnow()

    # last_reported
    df.last_reported = df.last_reported.apply(
        lambda x: datetime(1970, 1, 1) + timedelta(seconds=x)
    )

    # station_id
    df.station_id = df.station_id.astype("int16")

    # legacy_id
    df.legacy_id = df.legacy_id.astype("int16")

    # station_status
    df.station_status = df.station_status.astype("category")

    # is_installed
    df.is_installed = df.is_installed.astype("bool")

    # is_renting
    df.is_renting = df.is_renting.astype("bool")

    # is_returning
    df.is_returning = df.is_returning.astype("bool")

    # num_docks_available
    df.num_docks_available = df.num_docks_available.astype("int8")

    # num_docks_disabled
    df.num_docks_disabled = df.num_docks_disabled.astype("int8")

    # num_bikes_available
    df.num_bikes_available = df.num_bikes_available.astype("int8")

    # num_bikes_disabled
    df.num_bikes_disabled = df.num_bikes_disabled.astype("int8")

    # num_ebikes_available
    df.num_bikes_available = df.num_bikes_available.astype("int8")

    # eightd_has_available_keys
    df.eightd_has_available_keys = df.eightd_has_available_keys.astype("bool")

    return df

cleaned_df = clean_station_status(df)
cleaned_df.dtypes

station_status                     category
is_renting                             bool
eightd_has_available_keys              bool
num_docks_available                    int8
is_returning                           bool
is_installed                           bool
num_docks_disabled                     int8
num_bikes_disabled                     int8
station_id                            int16
num_bikes_available                    int8
num_ebikes_available                  int64
last_reported                datetime64[ns]
legacy_id                             int16
time_prepared                datetime64[ns]
dtype: object

In [142]:
cleaned_df.head(2)

,station_status,is_renting,eightd_has_available_keys,num_docks_available,is_returning,is_installed,num_docks_disabled,num_bikes_disabled,station_id,num_bikes_available,num_ebikes_available,last_reported,legacy_id,time_prepared
0,active,True,False,12,True,True,0,0,3,3,0,2023-03-03 13:24:47,3,2023-03-03 13:29:59.706658
1,active,True,False,12,True,True,0,0,4,7,0,2023-03-03 13:27:47,4,2023-03-03 13:29:59.706658


In [129]:
df.num_docks_available.value_counts()

0     72
7     31
3     30
9     28
6     28
5     27
1     26
10    24
8     24
12    22
2     21
4     20
15    17
13    16
11    16
16    10
17     8
14     7
18     6
19     5
21     4
20     3
24     2
22     2
26     2
32     1
29     1
Name: num_docks_available, dtype: int64

In [19]:
from datetime import datetime, timedelta
first_date = datetime(1970, 1, 1)
time_since = datetime.now() - first_date
seconds = int(time_since.total_seconds())
print(seconds)

1677825345


In [20]:
print(datetime(1970, 1, 1) + timedelta(seconds=seconds))

2023-03-03 06:35:45


In [148]:
from datetime import datetime, timedelta
local_path = f"data/stationdata/station_status-{datetime.now().year:4}_{datetime.now().month:02}_{datetime.now().day:02}_{datetime.now().hour:02}-{datetime.now().minute:02}.parquet"

local_path.rstrip("parquet").rstrip(".").lstrip("data/").lstrip("stationdata").lstrip("/")

'station_status-2023_03_03_08-42'